In [1]:
#for keras 
#kernel = tensorflow
import keras
from keras import layers
import tensorflow as tf

#packages for embedding
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
%matplotlib inline

from IPython.display import SVG
from urllib.request import urlretrieve
import zipfile
import collections

from datetime import datetime

import argparse

In [2]:
import tensorflow.compat.v1 as tf

In [3]:
import os
import pandas as pd
import numpy as np
import time
import timeit
import math

In [4]:
#data processing
#get data based on census tract
#based on downtown Seattle
#test data
os.chdir('G:/My Drive/2021/Bias')

test_matrice_reverse = pd.read_csv('test_matrice_reverse.csv', header=None).to_numpy()
Zero_index = pd.read_csv('Zero_index.csv', header=None).to_numpy()
#os.listsdir()

#land use data 
#obs
#obs_01
#zero index
'''
landuse_input_path = 'G:/My Drive/2021/Bias/census_block_data/'
land_use_obs = pd.read_csv('obs_landuse.csv', header=None).to_numpy()
land_use_01 =  pd.read_csv('obs01_landuse.csv', header=None).to_numpy()

land_use_zero_index = pd.read_csv('zero_index_landuse.csv',header=None).to_numpy()'''

"\nlanduse_input_path = 'G:/My Drive/2021/Bias/census_block_data/'\nland_use_obs = pd.read_csv('obs_landuse.csv', header=None).to_numpy()\nland_use_01 =  pd.read_csv('obs01_landuse.csv', header=None).to_numpy()\n\nland_use_zero_index = pd.read_csv('zero_index_landuse.csv',header=None).to_numpy()"

In [5]:
num_users, num_items = test_matrice_reverse[:, 1:].shape
num_obs = np.count_nonzero(test_matrice_reverse[:, 1:])

In [6]:
num_obs

112360

### AutoRec
[Source](https://github.com/gtshs2/Autorec)

In [7]:
#revised data processing code
#data format
#matrix user * item 
def convert_matrix2df(obs_matrix):
    num_users, num_items = obs_matrix.shape
    rating_val = obs_matrix.flatten()
    user_index = np.repeat(np.array(range(num_users)), 
                           repeats=num_items,
                           axis=0)
    item_index = np.tile(np.array(range(num_items)),
                         num_users)
    df = np.vstack([user_index, item_index, rating_val]).T
    df = np.delete(df, np.where(df[:, 2]==0), axis=0)
    return(df)

def dt_processing(obs_matrix, num_total_ratings, a, b, train_ratio):
    #fp = open(path + "ratings.dat")
    
    #notice input obs matrix should not cover the user/item index
    num_users, num_items = obs_matrix.shape
    num_total_ratings = np.count_nonzero(obs_matrix)

    user_train_set = set()
    user_test_set = set()
    item_train_set = set()
    item_test_set = set()

    R = obs_matrix.copy()
    mask_R = np.where(obs_matrix>0, 1, 0)
    C = np.where(obs_matrix>0, a, b)

    train_R = np.zeros((num_users, num_items))
    test_R = np.zeros((num_users, num_items))

    train_mask_R = np.zeros((num_users, num_items))
    test_mask_R = np.zeros((num_users, num_items))

    random_perm_idx = np.random.permutation(num_total_ratings)
    train_idx = random_perm_idx[0:int(num_total_ratings*train_ratio)]
    test_idx = random_perm_idx[int(num_total_ratings*train_ratio):]
    #print(train_idx)
    #print(test_idx)
    
    num_train_ratings = len(train_idx)
    num_test_ratings = len(test_idx)

    #lines = fp.readlines()
    #for line in lines:
        #user,item,rating,_ = line.split("::")
        #user_idx = int(user) - 1
        #item_idx = int(item) - 1 
        #mask_R[user_idx,item_idx] = 1
        #C[user_idx,item_idx] = a   
    
    revised_matrix = convert_matrix2df(obs_matrix)
    #print(revised_matrix)
    
    ''' Train '''
    for itr in train_idx:
        #line = lines[itr]
        #user,item,rating,_ = line.split("::")
        user_idx = int(revised_matrix[itr][0])
        item_idx = int(revised_matrix[itr][1])
        #print(user_idx, item_idx, revised_matrix[itr][2])
        train_R[user_idx,item_idx] = int(revised_matrix[itr][2])
        train_mask_R[user_idx,item_idx] = 1

        user_train_set.add(user_idx)
        item_train_set.add(item_idx)

    ''' Test '''
    for itr in test_idx:
        #line = lines[itr]
        #user, item, rating, _ = line.split("::")
        user_idx = int(revised_matrix[itr][0])
        item_idx = int(revised_matrix[itr][1])
        test_R[user_idx, item_idx] = int(revised_matrix[itr][2])
        test_mask_R[user_idx, item_idx] = 1

        user_test_set.add(user_idx)
        item_test_set.add(item_idx)

    return R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R,num_train_ratings,num_test_ratings,\
user_train_set,item_train_set,user_test_set,item_test_set

#test code
#R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R,
#num_train_ratings,num_test_ratings,\
#user_train_set,item_train_set,user_test_set,item_test_set
#test_matrix = np.ones([3,6])
#test_matrix[0][1] = test_matrix[0][1]-1
#for i in range(6):
#    test_matrix[2][i] = test_matrix[2][i]+i
#print(test_matrix)
#dt_processing(test_matrix, 17, 3, 6, 0.5)

In [8]:
#main code
class AutoRec():
    def __init__(self,sess,args,
                      num_users,num_items,
                      R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R,
                      num_train_ratings,num_test_ratings,
                      user_train_set, item_train_set, user_test_set, item_test_set,
                      result_path):

        self.sess = sess
        self.args = args

        self.num_users = num_users
        self.num_items = num_items

        self.R = R
        self.mask_R = mask_R
        self.C = C
        self.train_R = train_R
        self.train_mask_R = train_mask_R
        self.test_R = test_R
        self.test_mask_R = test_mask_R
        self.num_train_ratings = num_train_ratings
        self.num_test_ratings = num_test_ratings

        self.user_train_set = user_train_set
        self.item_train_set = item_train_set
        self.user_test_set = user_test_set
        self.item_test_set = item_test_set

        self.hidden_neuron = args.hidden_neuron
        self.train_epoch = args.train_epoch
        self.batch_size = args.batch_size
        self.num_batch = int(math.ceil(self.num_users / float(self.batch_size)))

        self.base_lr = args.base_lr
        self.optimizer_method = args.optimizer_method
        self.display_step = args.display_step
        self.random_seed = args.random_seed

        self.global_step = tf.Variable(0, trainable=False)
        self.decay_epoch_step = args.decay_epoch_step
        self.decay_step = self.decay_epoch_step * self.num_batch
        self.lr = tf.train.exponential_decay(self.base_lr, self.global_step,
                                                   self.decay_step, 0.96, staircase=True)
        self.lambda_value = args.lambda_value

        self.train_cost_list = []
        self.test_cost_list = []
        self.test_rmse_list = []

        self.result_path = result_path
        self.grad_clip = args.grad_clip

    def run(self):
        self.prepare_model()
        init = tf.global_variables_initializer()
        self.sess.run(init)
        for epoch_itr in range(self.train_epoch):
            self.train_model(epoch_itr)
            self.test_model(epoch_itr)
        self.make_records()

    def prepare_model(self):
        self.input_R = tf.placeholder(dtype=tf.float32, shape=[None, self.num_items], name="input_R")
        self.input_mask_R = tf.placeholder(dtype=tf.float32, shape=[None, self.num_items], name="input_mask_R")

        V = tf.get_variable(name="V", initializer=tf.truncated_normal(shape=[self.num_items, self.hidden_neuron],
                                         mean=0, stddev=0.03),dtype=tf.float32)
        W = tf.get_variable(name="W", initializer=tf.truncated_normal(shape=[self.hidden_neuron, self.num_items],
                                         mean=0, stddev=0.03),dtype=tf.float32)
        mu = tf.get_variable(name="mu", initializer=tf.zeros(shape=self.hidden_neuron),dtype=tf.float32)
        b = tf.get_variable(name="b", initializer=tf.zeros(shape=self.num_items), dtype=tf.float32)

        pre_Encoder = tf.matmul(self.input_R,V) + mu
        self.Encoder = tf.nn.sigmoid(pre_Encoder)
        pre_Decoder = tf.matmul(self.Encoder,W) + b
        self.Decoder = tf.identity(pre_Decoder)

        pre_rec_cost = tf.multiply((self.input_R - self.Decoder) , self.input_mask_R)
        rec_cost = tf.square(self.l2_norm(pre_rec_cost))
        pre_reg_cost = tf.square(self.l2_norm(W)) + tf.square(self.l2_norm(V))
        reg_cost = self.lambda_value * 0.5 * pre_reg_cost

        self.cost = rec_cost + reg_cost

        if self.optimizer_method == "Adam":
            optimizer = tf.train.AdamOptimizer(self.lr)
        elif self.optimizer_method == "RMSProp":
            optimizer = tf.train.RMSPropOptimizer(self.lr)
        else:
            raise ValueError("Optimizer Key ERROR")

        if self.grad_clip:
            gvs = optimizer.compute_gradients(self.cost)
            capped_gvs = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gvs]
            self.optimizer = optimizer.apply_gradients(capped_gvs, global_step=self.global_step)
        else:
            self.optimizer = optimizer.minimize(self.cost, global_step=self.global_step)

    def train_model(self,itr):
        start_time = time.time()
        random_perm_doc_idx = np.random.permutation(self.num_users)

        batch_cost = 0
        for i in range(self.num_batch):
            if i == self.num_batch - 1:
                batch_set_idx = random_perm_doc_idx[i * self.batch_size:]
            elif i < self.num_batch - 1:
                batch_set_idx = random_perm_doc_idx[i * self.batch_size : (i+1) * self.batch_size]

            _, Cost = self.sess.run(
                [self.optimizer, self.cost],
                feed_dict={self.input_R: self.train_R[batch_set_idx, :],
                           self.input_mask_R: self.train_mask_R[batch_set_idx, :]})

            batch_cost = batch_cost + Cost
        self.train_cost_list.append(batch_cost)

        if (itr+1) % self.display_step == 0:
            print ("Training //", "Epoch %d //" % (itr), " Total cost = {:.2f}".format(batch_cost),
               "Elapsed time : %d sec" % (time.time() - start_time))

    def test_model(self,itr):
        start_time = time.time()
        Cost,Decoder = self.sess.run(
            [self.cost,self.Decoder],
            feed_dict={self.input_R: self.test_R,
                       self.input_mask_R: self.test_mask_R})

        self.test_cost_list.append(Cost)

        if (itr+1) % self.display_step == 0:
            Estimated_R = Decoder.clip(min=1, max=5)
            unseen_user_test_list = list(self.user_test_set - self.user_train_set)
            unseen_item_test_list = list(self.item_test_set - self.item_train_set)

            for user in unseen_user_test_list:
                for item in unseen_item_test_list:
                    if self.test_mask_R[user,item] == 1: # exist in test set
                        Estimated_R[user,item] = 3

            pre_numerator = np.multiply((Estimated_R - self.test_R), self.test_mask_R)
            numerator = np.sum(np.square(pre_numerator))
            denominator = self.num_test_ratings
            RMSE = np.sqrt(numerator / float(denominator))

            self.test_rmse_list.append(RMSE)

            print ("Testing //", "Epoch %d //" % (itr), " Total cost = {:.2f}".format(Cost), " RMSE = {:.5f}".format(RMSE),
                   "Elapsed time : %d sec" % (time.time() - start_time))
            print ("=" * 100)

    def make_records(self):
        if not os.path.exists(self.result_path):
            os.makedirs(self.result_path)

        basic_info = self.result_path + "basic_info.txt"
        train_record = self.result_path + "train_record.txt"
        test_record = self.result_path + "test_record.txt"

        with open (train_record,'w') as f:
            f.write(str("Cost:"))
            f.write('\t')
            for itr in range(len(self.train_cost_list)):
                f.write(str(self.train_cost_list[itr]))
                f.write('\t')
            f.write('\n')

        with open (test_record,'w') as g:
            g.write(str("Cost:"))
            g.write('\t')
            for itr in range(len(self.test_cost_list)):
                g.write(str(self.test_cost_list[itr]))
                g.write('\t')
            g.write('\n')

            g.write(str("RMSE:"))
            for itr in range(len(self.test_rmse_list)):
                g.write(str(self.test_rmse_list[itr]))
                g.write('\t')
            g.write('\n')

        with open(basic_info,'w') as h:
            h.write(str(self.args))

    def l2_norm(self,tensor):
        return tf.sqrt(tf.reduce_sum(tf.square(tensor)))

In [100]:
os.chdir('Bias/')

In [104]:
#os.listdir()

In [9]:
## main
current_time = time.time()

parser = argparse.ArgumentParser(description='I-AutoRec ')
parser.add_argument('--hidden_neuron', type=int, default=500)
parser.add_argument('--lambda_value', type=float, default=1)

parser.add_argument('--train_epoch', type=int, default=2000)
parser.add_argument('--batch_size', type=int,default=100)

parser.add_argument('--optimizer_method', choices=['Adam','RMSProp'],default='Adam')
parser.add_argument('--grad_clip', type=bool,default=False)
parser.add_argument('--base_lr', type=float, default=1e-3)
parser.add_argument('--decay_epoch_step', type=int, default=0.1, help="decay the learning rate for each n epochs")

parser.add_argument('--random_seed', type=int, default=1000)
parser.add_argument('--display_step', type=int, default=1)

#args = parser.parse_args()
#tf.set_random_seed(args.random_seed)
args, unknown  = parser.parse_known_args()
tf.random.set_random_seed(args.random_seed)
np.random.seed(args.random_seed)

#data_name = 'ml-1m'; num_users = 6040; num_items = 3952; num_total_ratings = 1000209; train_ratio = 0.9
#path = "./data/%s" % data_name + "/"

#result_path = './results/' + data_name + '/' + 
#str(args.random_seed) + '_' + str(args.optimizer_method) + 
#'_' + str(args.base_lr) + "_" + str(current_time)+"/"

#R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R,num_train_ratings,num_test_ratings,\
#user_train_set,item_train_set,user_test_set,item_test_set \
#    = read_rating(path, num_users, num_items,num_total_ratings, 1, 0, train_ratio)

#dt_processing(obs_matrix, num_total_ratings, a, b, train_ratio)
R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R,num_train_ratings,num_test_ratings,\
user_train_set,item_train_set,user_test_set,item_test_set \
    = dt_processing(test_matrice_reverse[:, 1:], num_obs, 1, 0, 0.9)
result_path = './results_decayrevised/'+'test_dt_obsdata'+'/'+ str(args.random_seed) + '_' + str(args.optimizer_method) + '_' + str(args.base_lr) + "_" + str(current_time)+"/"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
with tf.Session(config=config) as sess:
    AutoRec = AutoRec(sess,args,
                      num_users,num_items,
                      R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R,num_train_ratings,num_test_ratings,
                      user_train_set, item_train_set, user_test_set, item_test_set,
                      result_path)
    AutoRec.run()

Training // Epoch 0 //  Total cost = 781233.06 Elapsed time : 3 sec
Testing // Epoch 0 //  Total cost = 70931.76  RMSE = 2.50097 Elapsed time : 1 sec
Training // Epoch 1 //  Total cost = 684081.24 Elapsed time : 3 sec
Testing // Epoch 1 //  Total cost = 70020.67  RMSE = 2.48572 Elapsed time : 0 sec
Training // Epoch 2 //  Total cost = 644883.70 Elapsed time : 3 sec
Testing // Epoch 2 //  Total cost = 69403.80  RMSE = 2.47515 Elapsed time : 1 sec
Training // Epoch 3 //  Total cost = 621724.32 Elapsed time : 3 sec
Testing // Epoch 3 //  Total cost = 69209.98  RMSE = 2.47267 Elapsed time : 1 sec
Training // Epoch 4 //  Total cost = 606353.36 Elapsed time : 3 sec
Testing // Epoch 4 //  Total cost = 69115.25  RMSE = 2.47164 Elapsed time : 1 sec
Training // Epoch 5 //  Total cost = 596133.83 Elapsed time : 3 sec
Testing // Epoch 5 //  Total cost = 69003.58  RMSE = 2.46970 Elapsed time : 1 sec
Training // Epoch 6 //  Total cost = 589501.29 Elapsed time : 3 sec
Testing // Epoch 6 //  Total cos

Training // Epoch 33 //  Total cost = 576269.65 Elapsed time : 3 sec
Testing // Epoch 33 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 34 //  Total cost = 576269.61 Elapsed time : 3 sec
Testing // Epoch 34 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 35 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 35 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 36 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 36 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 37 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 37 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 38 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 38 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 39 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 39

Training // Epoch 66 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 66 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 67 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 67 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 68 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing // Epoch 68 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 69 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 69 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 70 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing // Epoch 70 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 71 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 71 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 72 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 72

Training // Epoch 99 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing // Epoch 99 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 100 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing // Epoch 100 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 101 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 101 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 102 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 102 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 103 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 103 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 104 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 104 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 105 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing 

Testing // Epoch 131 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 132 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 132 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 133 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing // Epoch 133 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 134 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 134 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 135 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 135 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 136 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 136 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 137 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 137 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time 

...

Training // Epoch 1841 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1841 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1842 //  Total cost = 576269.57 Elapsed time : 3 sec
Testing // Epoch 1842 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1843 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1843 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1844 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1844 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1845 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1845 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1846 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1846 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1847 //  Total cost = 576269.59 Elapsed time :

Training // Epoch 1873 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1873 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1874 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1874 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1875 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1875 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1876 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 1876 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1877 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing // Epoch 1877 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1878 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1878 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1879 //  Total cost = 576269.60 Elapsed time :

Training // Epoch 1905 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1905 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1906 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1906 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1907 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1907 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1908 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1908 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1909 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1909 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1910 //  Total cost = 576269.60 Elapsed time : 3 sec
Testing // Epoch 1910 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1911 //  Total cost = 576269.58 Elapsed time :

Training // Epoch 1937 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1937 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1938 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing // Epoch 1938 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1939 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1939 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1940 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing // Epoch 1940 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1941 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1941 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1942 //  Total cost = 576269.58 Elapsed time : 3 sec
Testing // Epoch 1942 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1943 //  Total cost = 576269.58 Elapsed time :

Training // Epoch 1969 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1969 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1970 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1970 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1971 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1971 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1972 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1972 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1973 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1973 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1974 //  Total cost = 576269.59 Elapsed time : 3 sec
Testing // Epoch 1974 //  Total cost = 68795.85  RMSE = 2.46676 Elapsed time : 1 sec
Training // Epoch 1975 //  Total cost = 576269.59 Elapsed time :

In [88]:
#data handling 
#original data
def read_rating(path, num_users, num_items, num_total_ratings, a, b, train_ratio):
    fp = open(path + "ratings.dat")

    user_train_set = set()
    user_test_set = set()
    item_train_set = set()
    item_test_set = set()

    R = np.zeros((num_users,num_items))
    mask_R = np.zeros((num_users, num_items))
    C = np.ones((num_users, num_items)) * b

    train_R = np.zeros((num_users, num_items))
    test_R = np.zeros((num_users, num_items))

    train_mask_R = np.zeros((num_users, num_items))
    test_mask_R = np.zeros((num_users, num_items))

    random_perm_idx = np.random.permutation(num_total_ratings)
    train_idx = random_perm_idx[0:int(num_total_ratings*train_ratio)]
    test_idx = random_perm_idx[int(num_total_ratings*train_ratio):]

    num_train_ratings = len(train_idx)
    num_test_ratings = len(test_idx)

    lines = fp.readlines()
    for line in lines:
        user,item,rating,_ = line.split("::")
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        R[user_idx,item_idx] = int(rating)
        mask_R[user_idx,item_idx] = 1
        C[user_idx,item_idx] = a

    ''' Train '''
    for itr in train_idx:
        line = lines[itr]
        user,item,rating,_ = line.split("::")
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        train_R[user_idx,item_idx] = int(rating)
        train_mask_R[user_idx,item_idx] = 1

        user_train_set.add(user_idx)
        item_train_set.add(item_idx)

    ''' Test '''
    for itr in test_idx:
        line = lines[itr]
        user, item, rating, _ = line.split("::")
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        test_R[user_idx, item_idx] = int(rating)
        test_mask_R[user_idx, item_idx] = 1

        user_test_set.add(user_idx)
        item_test_set.add(item_idx)

    return R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R,num_train_ratings,num_test_ratings,\
user_train_set,item_train_set,user_test_set,item_test_set